In [1]:
import cv2
import numpy as np
from pathlib import Path

In [2]:
path2 = Path('../../data/exp_openCV/DJI_20241014141733_0002_D.JPG').resolve()
path3 = Path('../../data/exp_openCV/DJI_20241014141735_0003_D.JPG').resolve()

In [3]:
# Caricare le immagini
img_ref = cv2.imread(str(path2), cv2.IMREAD_GRAYSCALE)
img_to_align = cv2.imread(str(path3), cv2.IMREAD_GRAYSCALE)


# Creare il detector ORB
orb = cv2.ORB_create(5000)


# Trovare i punti chiave e descrittori
kp1, des1 = orb.detectAndCompute(img_ref, None)
kp2, des2 = orb.detectAndCompute(img_to_align, None)


# Creare il matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)


# Trovare le corrispondenze
matches = bf.match(des1, des2)


# Ordinare per distanza
matches = sorted(matches, key=lambda x: x.distance)


# Prendere i migliori 50 punti
matches = matches[:50]


In [4]:

# Estrarre i punti di corrispondenza
src_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)


In [5]:

# Calcolare la trasformazione omografica
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)


In [6]:

# Applicare la trasformazione all'immagine da allineare
h, w = img_ref.shape
aligned_img = cv2.warpPerspective(cv2.imread(str(path3)), H, (w, h))


In [7]:

# Salvare o mostrare l'immagine allineata
cv2.imwrite("immagine_allineata.jpg", aligned_img)
# cv2.imshow("Allineata", aligned_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [8]:


# Carica l'immagine originale e quella allineata
original_img = cv2.imread(str(path2))  # Sostituisci con il nome corretto
aligned_img = cv2.imread("immagine_allineata.jpg")

# Assicurati che entrambe le immagini abbiano le stesse dimensioni
if original_img.shape != aligned_img.shape:
    aligned_img = cv2.resize(aligned_img, (original_img.shape[1], original_img.shape[0]))

# Sovrapposizione con trasparenza (alpha blend)
alpha = 0.6  # Livello di trasparenza dell'immagine allineata
overlay = cv2.addWeighted(original_img, 1 - alpha, aligned_img, alpha, 0)

# Salva l'immagine sovrapposta
cv2.imwrite("sovrapposta.jpg", overlay)

# Mostra l'immagine sovrapposta
cv2.imshow("Immagine Sovrapposta", overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/mauro/.local/lib/python3.10/site-packages/cv2/qt/plugins"
